# Deadlock Analytics - Main Notebook

## 1. Setup and Imports

In [ ]:
import pandas as pd
import json
import os
import plotly.express as px
from dotenv import load_dotenv

# Load env vars if needed
load_dotenv()

# Settings
pd.set_option('display.max_columns', None)

## 2. Load Data
Loading the most recent `active_matches` file from `../data/raw`.

In [ ]:
def load_latest_matches():
    raw_dir = os.path.join("..", "data", "raw")
    files = [f for f in os.listdir(raw_dir) if f.startswith("active_matches") and f.endswith(".json")]
    if not files:
        raise FileNotFoundError("No match files found!")
    
    # Sort by name (timestamp is in name) to get latest
    latest_file = sorted(files)[-1]
    print(f"Loading: {latest_file}")
    
    with open(os.path.join(raw_dir, latest_file), 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

raw_data = load_latest_matches()
print(f"Loaded {len(raw_data)} matches")

## 3. Data Processing
Converting the list of matches into a pandas DataFrame.

In [ ]:
df_matches = pd.DataFrame(raw_data)
df_matches.head()

### 3.1 Player Level Data
The `players` column contains a list of player dicts. Let's explode requests to get player-level stats.

In [ ]:
# Normalize/explode 'players' column
df_players = df_matches.explode('players').reset_index(drop=True)

# Normalize the 'players' column into separate columns
players_norm = pd.json_normalize(df_players['players'])

# Combine with match info (repeat match metadata for each player)
df_full = pd.concat([df_players.drop(columns=['players']), players_norm], axis=1)

df_full.head()

## 4. Basic Analysis
### 4.1 Matches by Mode

In [ ]:
if 'match_mode' in df_matches.columns:
    print(df_matches['match_mode'].value_counts())

### 4.2 Hero Popularity (in this sample)

In [ ]:
if 'hero_id' in df_full.columns:
    hero_counts = df_full['hero_id'].value_counts().reset_index()
    hero_counts.columns = ['hero_id', 'count']
    
    fig = px.bar(hero_counts, x='hero_id', y='count', title='Hero Popularity in Active Matches')
    fig.show()